In [1]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import hopsworks
import pandas as pd
import numpy as np

In [2]:
#read the wine quality data set
wine_df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv")
wine_df

type  fixed acidity  volatile acidity  citric acid  residual sugar  \
0     white            7.0             0.270         0.36            20.7   
1     white            6.3             0.300         0.34             1.6   
2     white            8.1             0.280         0.40             6.9   
3     white            7.2             0.230         0.32             8.5   
4     white            7.2             0.230         0.32             8.5   
...     ...            ...               ...          ...             ...   
6492    red            6.2             0.600         0.08             2.0   
6493    red            5.9             0.550         0.10             2.2   
6494    red            6.3             0.510         0.13             2.3   
6495    red            5.9             0.645         0.12             2.0   
6496    red            6.0             0.310         0.47             3.6   

      chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
0         0.045                 45.0                 170.0  1.00100  3.00   
1         0.049                 14.0                 132.0  0.99400  3.30   
2         0.050                 30.0                  97.0  0.99510  3.26   
3         0.058                 47.0                 186.0  0.99560  3.19   
4         0.058                 47.0                 186.0  0.99560  3.19   
...         ...                  ...                   ...      ...   ...   
6492      0.090                 32.0                  44.0  0.99490  3.45   
6493      0.062                 39.0                  51.0  0.99512  3.52   
6494      0.076                 29.0                  40.0  0.99574  3.42   
6495      0.075                 32.0                  44.0  0.99547  3.57   
6496      0.067                 18.0                  42.0  0.99549  3.39   

      sulphates  alcohol  quality  
0          0.45      8.8        6  
1          0.49      9.5        6  
2          0.44     10.1        6  
3          0.40      9.9        6  
4          0.40      9.9        6  
...         ...      ...      ...  
6492       0.58     10.5        5  
6493        NaN     11.2        6  
6494       0.75     11.0        6  
6495       0.71     10.2        5  
6496       0.66     11.0        6  

[6497 rows x 13 columns]

In [3]:
wine_df['type'].value_counts()

type
white    4898
red      1599
Name: count, dtype: int64

In [4]:
numerical_col = list(wine_df.columns[1:11])
categorical_col = ["type", "quality"]
wine_df[numerical_col] = wine_df[numerical_col].fillna(wine_df[numerical_col].mean())
wine_df["type"] = wine_df["type"].fillna("unknown")

In [5]:
#-Transform categorical variables into numerical variables
wine_df['type'].replace(['white', 'red', 'unknown'], [0, 1, 2], inplace=True)
wine_df

type  fixed acidity  volatile acidity  citric acid  residual sugar  \
0        0            7.0             0.270         0.36            20.7   
1        0            6.3             0.300         0.34             1.6   
2        0            8.1             0.280         0.40             6.9   
3        0            7.2             0.230         0.32             8.5   
4        0            7.2             0.230         0.32             8.5   
...    ...            ...               ...          ...             ...   
6492     1            6.2             0.600         0.08             2.0   
6493     1            5.9             0.550         0.10             2.2   
6494     1            6.3             0.510         0.13             2.3   
6495     1            5.9             0.645         0.12             2.0   
6496     1            6.0             0.310         0.47             3.6   

      chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
0         0.045                 45.0                 170.0  1.00100  3.00   
1         0.049                 14.0                 132.0  0.99400  3.30   
2         0.050                 30.0                  97.0  0.99510  3.26   
3         0.058                 47.0                 186.0  0.99560  3.19   
4         0.058                 47.0                 186.0  0.99560  3.19   
...         ...                  ...                   ...      ...   ...   
6492      0.090                 32.0                  44.0  0.99490  3.45   
6493      0.062                 39.0                  51.0  0.99512  3.52   
6494      0.076                 29.0                  40.0  0.99574  3.42   
6495      0.075                 32.0                  44.0  0.99547  3.57   
6496      0.067                 18.0                  42.0  0.99549  3.39   

      sulphates  alcohol  quality  
0      0.450000      8.8        6  
1      0.490000      9.5        6  
2      0.440000     10.1        6  
3      0.400000      9.9        6  
4      0.400000      9.9        6  
...         ...      ...      ...  
6492   0.580000     10.5        5  
6493   0.531215     11.2        6  
6494   0.750000     11.0        6  
6495   0.710000     10.2        5  
6496   0.660000     11.0        6  

[6497 rows x 13 columns]

In [6]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
# Use mutual information for feature selection 
selector = SelectKBest(mutual_info_classif, k=10)
X = wine_df.drop('quality', axis=1)
y = wine_df['quality']
X_selected = selector.fit_transform(X, y)

# Get selected feature indices
selected_indices = selector.get_support()

# Get the selected feature names
selected_features = X.columns[selected_indices]
dropped_features = X.columns[~selected_indices]

# Print the selected features
print("Selected features:", selected_features)
print("Dropped features:", dropped_features)

Selected features: Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'sulphates', 'alcohol'],
      dtype='object')
Dropped features: Index(['type', 'pH'], dtype='object')


In [7]:
wine_df = wine_df.drop(dropped_features, axis=1)
wine_df

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0             0.270         0.36            20.7      0.045   
1               6.3             0.300         0.34             1.6      0.049   
2               8.1             0.280         0.40             6.9      0.050   
3               7.2             0.230         0.32             8.5      0.058   
4               7.2             0.230         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
6492            6.2             0.600         0.08             2.0      0.090   
6493            5.9             0.550         0.10             2.2      0.062   
6494            6.3             0.510         0.13             2.3      0.076   
6495            5.9             0.645         0.12             2.0      0.075   
6496            6.0             0.310         0.47             3.6      0.067   

      free sulfur dioxide  total sulfur dioxide  density  sulphates  alcohol  \
0                    45.0                 170.0  1.00100   0.450000      8.8   
1                    14.0                 132.0  0.99400   0.490000      9.5   
2                    30.0                  97.0  0.99510   0.440000     10.1   
3                    47.0                 186.0  0.99560   0.400000      9.9   
4                    47.0                 186.0  0.99560   0.400000      9.9   
...                   ...                   ...      ...        ...      ...   
6492                 32.0                  44.0  0.99490   0.580000     10.5   
6493                 39.0                  51.0  0.99512   0.531215     11.2   
6494                 29.0                  40.0  0.99574   0.750000     11.0   
6495                 32.0                  44.0  0.99547   0.710000     10.2   
6496                 18.0                  42.0  0.99549   0.660000     11.0   

      quality  
0           6  
1           6  
2           6  
3           6  
4           6  
...       ...  
6492        5  
6493        6  
6494        6  
6495        5  
6496        6  

[6497 rows x 11 columns]

In [8]:
wine_df.columns = wine_df.columns.str.replace(' ', '_')
wine_df

fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0               7.0             0.270         0.36            20.7      0.045   
1               6.3             0.300         0.34             1.6      0.049   
2               8.1             0.280         0.40             6.9      0.050   
3               7.2             0.230         0.32             8.5      0.058   
4               7.2             0.230         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
6492            6.2             0.600         0.08             2.0      0.090   
6493            5.9             0.550         0.10             2.2      0.062   
6494            6.3             0.510         0.13             2.3      0.076   
6495            5.9             0.645         0.12             2.0      0.075   
6496            6.0             0.310         0.47             3.6      0.067   

      free_sulfur_dioxide  total_sulfur_dioxide  density  sulphates  alcohol  \
0                    45.0                 170.0  1.00100   0.450000      8.8   
1                    14.0                 132.0  0.99400   0.490000      9.5   
2                    30.0                  97.0  0.99510   0.440000     10.1   
3                    47.0                 186.0  0.99560   0.400000      9.9   
4                    47.0                 186.0  0.99560   0.400000      9.9   
...                   ...                   ...      ...        ...      ...   
6492                 32.0                  44.0  0.99490   0.580000     10.5   
6493                 39.0                  51.0  0.99512   0.531215     11.2   
6494                 29.0                  40.0  0.99574   0.750000     11.0   
6495                 32.0                  44.0  0.99547   0.710000     10.2   
6496                 18.0                  42.0  0.99549   0.660000     11.0   

      quality  
0           6  
1           6  
2           6  
3           6  
4           6  
...       ...  
6492        5  
6493        6  
6494        6  
6495        5  
6496        6  

[6497 rows x 11 columns]

Write the features to the feature store as a Feature Group

In [12]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/212121
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
wine_fg = fs.get_or_create_feature_group(
    name="wine_quality",
    version=1,
    primary_key=list(wine_df.columns), 
    description="Wine quality dataset")
wine_fg.insert(wine_df)